In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,TimeSeriesSplit

In [ ]:
params = {'num_leaves': 491,
          'colsample_bytree': 0.75,
          'subsample': 0.75,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.45,
          'reg_lambda': 0.65,
          'random_state': 47,
          'n_jobs':-1,
          'n_estimators':10000,
         }

In [ ]:
def lgb_model(params=None,cv=KFold(5),X=None,X_test=None,columns=None,early_stopping_rounds=200, \
              cols=None,categorical_feature=None,group_feature=None):


    #folds = StratifiedKFold(n_splits=NFOLDS)
    folds =cv
    if cols==None:
        columns = list(X.columns)
    else:
        columns=cols
    if group_feature:
        columns.remove(group_feature)
        splits = folds.split(X[columns], y,groups=X[group_feature])
    else:
        splits = folds.split(X[columns], y)
    y_oof = np.zeros(X.shape[0])
    score = 0


    clfs=[]
    for fold_n, (train_index, valid_index) in enumerate(splits):
        X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]


        clf = lgb.LGBMClassifier(**params)
        clf.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_valid,y_valid)], \
                 eval_metric='auc', verbose=100, early_stopping_rounds=early_stopping_rounds,categorical_feature=categorical_feature)


        y_pred_valid = clf.predict_proba(X_valid)[:,1]
        y_oof[valid_index] = y_pred_valid
        print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")

        score += roc_auc_score(y_valid, y_pred_valid) / 5

        del X_train, X_valid, y_train, y_valid
        gc.collect()
        clfs.append(clf)

    print(f"\nMean AUC = {score}")
    print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")
    return clfs

In [ ]:
clfs=lgb_model(params=params,cv=StratifiedKFold(5),X=X,X_test=X_test,early_stopping_rounds=200,categorical_feature=cat)

In [ ]:
clfs=lgb_model(params=params,cv=KFold(5),X=X,X_test=X_test,early_stopping_rounds=200,cols=None,categorical_feature=cat)

In [ ]:
clfs=lgb_model(params=params,cv=TimeSeriesSplit(5),X=X,X_test=X_test,early_stopping_rounds=200,cols=None,categorical_feature=cat)

In [ ]:
clfs=lgb_model(params=params,cv=GroupKFold(5),X=X,X_test=X_test,early_stopping_rounds=200,cols=None,categorical_feature=cat,group_feature='DT_M')